# 📌 상품 정보 크롤링  
* urllib 라이브러리 사용해 파일 다운로드

### ✏️ 상품 페이지 목록 받아오기

In [ ]:
# 라이브러리 읽어들이기
import requests
import re
import os
from bs4 import BeautifulSoup as bs

# 파일 주소 크롤링
def bs_parsing(url):
  response = requests.get(root_url + url)
  soup = bs(response.text, "html.parser")
  elements = soup.select("table#flisttable td > a")
  
  # 주소 담을 리스트
  page_list = []
  
  # 상세 페이지 주소 리스트화
  for (idx, el) in enumerate(elements, 1):
    page_list.append(root_url + el.attrs["href"].replace("%2F", "/"))
    
  return page_list

# 리스트 값을 txt 파일에 한 줄씩 저장
def save_text_file(page_list):
  f = open("C:/Users/sodud/study/ssafy_spec_pjt/subtitle/data/page_url_list.txt", "w", encoding="UTF8")
  for page in page_list:
    f.write(page)
    f.write("\n")
  f.close
  ("[SUCCESS] 자막 페이지 목록 저장")

# 자막 사이트
root_url = "https://kitsunekko.net"

# 일본어 자막 목록 페이지
page_url = "/dirlist.php?dir=subtitles/japanese/"

# 자막 페이지 목록 리스트로 저장
page_list = bs_parsing(page_url)

# 리스트를 txt 파일로 저장
save_text_file(page_list)

### ✏️ 상품 정보 엑셀로 저장

In [84]:
# import chromedriver_autoinstaller
# chromedriver_autoinstaller.install()

import requests
import re
import os
from selenium import webdriver
from bs4 import BeautifulSoup
from openpyxl import Workbook
import pymysql

# DB 연결용
conn = pymysql.connect(host='k7e201.p.ssafy.io',
                       user='abuyz7e201',
                       password='a-buy-z-e201',
                       db='wkdbf',
                       charset='utf8')


sql = "SELECT * FROM products"

with conn:
    with conn.cursor() as cur:
        cur.execute(sql)
        result = cur.fetchall()
        for data in result:
            print(data)


# wb = Workbook()
# ws = wb.create_sheet('상품정보')
# # 컬럼 명
# ws.append([ '이름', '가격', '설명 이미지', '상품 상태', '할인율', '리뷰 평점', '배송료', '상품사진1','상품사진2','상품사진3'])

# 임시 url
temp_url= "https://www.lotteon.com/p/product/LO1987818462?areaCode=AD_CPC&entryPoint=ad&dp_infw_cd=CASFC01200000&clickId=C33295679634"

webdriver_options=webdriver.ChromeOptions()
webdriver_options.add_argument("--start-maximized")
web_driver = webdriver.Chrome(options=webdriver_options)
web_driver.get(temp_url)
web_driver.implicitly_wait(5)
html = web_driver.page_source
soup = BeautifulSoup(html, 'html.parser')
# temp = soup.find('div', class_='productName')



 

product_name = soup.find('div', class_='productName').text
product_price = soup.find('div', class_='listPrice').select_one(":nth-child(2)").text.replace(" ","").replace("\n","").replace(",","")
product_price = int(product_price)
des_picture = soup.find('iframe', class_='m2-prd-frame')['src']


# origin = soup.find('div', class_='productName').text
# status = soup.find('div', class_='productName').text
# producer = soup.find('div', class_='productName').text
status = '새상품'


discount_rate = soup.find('span', class_='discountRate').text.replace("할인율","").replace(" ","").replace("%","")
discount_rate = int(discount_rate)
review_rating = soup.find('a', class_='staring').em.text # 5.0
delivery_fee = soup.find_all('li', class_='soonInfoList')[1].text

# 배송비 무료 > int 변환
val = "무료" in delivery_fee
if val:
    delivery_fee = 0
else:
    delivery_fee = 3000


############### 이미지 ################
picture_list = soup.find_all('img', class_='loaded')
# picture_list
# for img in picture_list:
#     print(img['src'])

# for idx in range(1,4):
#     print(picture_list[idx]['src'])
#     print(idx)
product_thumbnail_1 = picture_list[1]['src']
product_thumbnail_2 = picture_list[2]['src']
product_thumbnail_3 = picture_list[3]['src']


# ws.append([product_name, product_price, des_picture, status, discount_rate, review_rating, delivery_fee, product_thumbnail_1, product_thumbnail_2, product_thumbnail_3 ])
# web_driver.implicitly_wait(3)

# wb.save('temp2.xlsx')

(1, datetime.datetime(2022, 11, 1, 0, 21, 41), 0, 'https://plussimage.lotte.com/cdnimage/ismsimage/GOODSDTLIMG/20220121/10/3e80b258-757c-42a9-bf21-14dd926a0752.jpg', 20, '에어 맥스 SC CW4555-002', 89000, None, 4.8, 'selling', 1, 8, None)
(2, datetime.datetime(2022, 11, 1, 0, 21, 41), 0, 'https://ai.esmplus.com/mc92931/%EC%BD%94%EC%8A%A4%ED%8A%B8%EC%BD%94/costco-604243/1.jpg', 0, '델몬트 스퀴즈 사과 오렌지 에이드 240ml x 30 x 2팩', 22890, None, 0.0, 'selling', 1, 5, None)
